<a href="https://colab.research.google.com/github/aakhterov/Python_practice/blob/master/SQL_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [2]:
%load_ext sql

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
def prepare_data_from_csv(path_to_csv: str) -> str:
  df = pd.read_csv(path_to_csv)
  return ','.join([str(tuple(row)) for row in df.values.tolist()])

In [4]:
%sql sqlite:////content/drive/MyDrive/sqltraining.sqlite

# Task #1 Duplicate Transactions

**Problem:**

The duplicate_transactions table contains transaction_id, timestamp, price and department.

Address these four questions:

1. How many duplicate records are there? For instance, if Row 1, and Row 2 and Row 3 contain the same values, then there are two duplicate records.

2. How many unique records have duplications?

3. Remove duplicate records, only preserving the unique records.

4. Which department has the highest duplicate records? Return the department name and count of duplicate records. Assume the possibility that multiple departments
could have the same highest count.

## 1.1. Creating tables and inserting values

In [6]:
%sql drop table duplicate_transactions

 * sqlite:////content/drive/MyDrive/sqltraining.sqlite
Done.


[]

In [7]:
%%sql
CREATE TABLE duplicate_transactions (
transaction_id VARCHAR,
timestamp INTEGER,
price INTEGER,
department VARCHAR);

 * sqlite:////content/drive/MyDrive/sqltraining.sqlite
Done.


[]

In [9]:
data = prepare_data_from_csv('/content/drive/MyDrive/Colab Notebooks/Data/SQL_training/duplicate_transactions/duplicate_transactions.csv')
data

"('qsllwzfgsu', 815725, 67, 'Movies'),('ldfdvwgsxa', 863206, 18, 'Groceries'),('glvkzgxobb', 846203, 58, 'Computer'),('gnrowryred', 865632, 48, 'Computer'),('wulekumebr', 871200, 47, 'Music'),('mudonayuog', 865647, 26, 'Groceries'),('xycfertiwm', 835662, 37, 'Music'),('jezmgvkcfh', 883012, 50, 'Groceries'),('wldyhjgwbm', 826626, 85, 'Books'),('jwqmpbxuhg', 830255, 62, 'Music'),('wwbvovukzk', 883079, 93, 'Movies'),('yqrliffwln', 817766, 98, 'MISC'),('dbqbjeupcc', 811894, 95, 'Groceries'),('ugbdqvafok', 867874, 98, 'Movies'),('drvvfsukzz', 820355, 70, 'Music'),('dxzwnbyaew', 866978, 59, 'Books'),('hvogwqqomb', 879565, 19, 'Movies'),('tjtxjmckvb', 815631, 28, 'MISC'),('umrbdqjdim', 857324, 31, 'Movies'),('qefbrneymv', 824134, 27, 'Music'),('xaakybfcib', 803316, 76, 'Computer'),('xzjzlwrsgr', 808747, 59, 'MISC'),('jukrlmgcqy', 874423, 26, 'Books'),('qldqkrteql', 842964, 57, 'Computer'),('xparscjhpi', 870162, 82, 'Computer'),('hqaobxlrcj', 879923, 45, 'MISC'),('drjbyjkdth', 889833, 49, 'Mus

In [10]:
%%sql insert into duplicate_transactions values {data}

 * sqlite:////content/drive/MyDrive/sqltraining.sqlite
580 rows affected.


[]

## 1.2. Look at the tables

In [15]:
%sql select * from duplicate_transactions limit 1, 5

 * sqlite:////content/drive/MyDrive/sqltraining.sqlite
Done.


transaction_id,timestamp,price,department
ldfdvwgsxa,863206,18,Groceries
glvkzgxobb,846203,58,Computer
gnrowryred,865632,48,Computer
wulekumebr,871200,47,Music
mudonayuog,865647,26,Groceries


## 1.3. Solution

1. How many duplicate records are there? For instance, if Row 1, and Row 2 and Row 3 contain the same values, then there are two duplicate records.

In [24]:
%%sql
select sum(c) as duplicates_count from
(
  select count(*)-1 as c
  from duplicate_transactions
  group by transaction_id, timestamp, price, department
  having count(*) > 1
)

 * sqlite:////content/drive/MyDrive/sqltraining.sqlite
Done.


duplicates_count
80


2. How many unique records have duplications?

In [25]:
%%sql
select count(transaction_id) as records_have_duplications from
(
  select transaction_id
  from duplicate_transactions
  group by transaction_id, timestamp, price, department
  having count(*) > 1
)

 * sqlite:////content/drive/MyDrive/sqltraining.sqlite
Done.


records_have_duplications
47


3. Remove duplicate records, only preserving the unique records.

In [31]:
%%sql
select distinct *
from duplicate_transactions

 * sqlite:////content/drive/MyDrive/sqltraining.sqlite
Done.


transaction_id,timestamp,price,department
qsllwzfgsu,815725,67,Movies
ldfdvwgsxa,863206,18,Groceries
glvkzgxobb,846203,58,Computer
gnrowryred,865632,48,Computer
wulekumebr,871200,47,Music
mudonayuog,865647,26,Groceries
xycfertiwm,835662,37,Music
jezmgvkcfh,883012,50,Groceries
wldyhjgwbm,826626,85,Books
jwqmpbxuhg,830255,62,Music


4. Which department has the highest duplicate records? Return the department name and count of duplicate records. Assume the possibility that multiple departments could have the same highest count.

In [38]:
%%sql
select department, s from
(
  select department, sum(c) as s from
    (
      select department, count(*) as c
      from duplicate_transactions
      group by transaction_id, timestamp, price, department
      having count(*) > 1
    )
  group by department
)
where s =
(
  select max(s) from
    (
      select department, sum(c) as s from
        (
          select department, count(*) as c
          from duplicate_transactions
          group by transaction_id, timestamp, price, department
          having count(*) > 1
        )
      group by department
    )
)

 * sqlite:////content/drive/MyDrive/sqltraining.sqlite
Done.


department,s
Music,33
